# Advanced `pandas`

In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd
import re

## Split-apply-combine

In [3]:
titanic_train = pd.read_csv("train.csv", index_col="PassengerId")
titanic_test = pd.read_csv("test.csv", index_col="PassengerId")
titanic = pd.concat([titanic_train, titanic_test], sort=False)

In [4]:
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Scalar output

In [ ]:
class_groups = titanic.groupby("Pclass")

In [ ]:
class_groups

In [ ]:
class_groups.mean()

In [ ]:
age_groups = titanic.Parch.groupby((5 + 10*(titanic.Age//10)))

In [ ]:
age_groups

In [ ]:
age_groups.mean() # Note index name

In [ ]:
age_groups_npy = titanic.Parch.groupby((5 + 10*(titanic.Age//10)).values)

In [ ]:
age_groups_npy.mean()

In [ ]:
age_groups_multi = titanic.Parch.groupby([(5 + 10*(titanic.Age//10)), titanic.Pclass])

In [ ]:
age_groups_multi = titanic.Parch.groupby([titanic.Pclass, (5 + 10*(titanic.Age//10))])

In [ ]:
age_groups_multi

In [ ]:
age_groups_multi.mean()

In [ ]:
age_groups_multi.mean().unstack()

In [ ]:
age_groups_mixed = titanic.groupby([(5 + 10*(titanic.Age//10)), "Pclass"])

In [ ]:
titanic

In [ ]:
5 + 10*(titanic.Age//10)

In [ ]:
age_groups_mixed.Parch.mean()#.unstack()

In [ ]:
print(age_groups_mixed.Parch.mean())

# Series output

In [ ]:
class_groups = titanic.groupby("Pclass") # Nothing is calculated yet

In [ ]:
n = titanic.groupby(["Pclass", "Sex"]).Age.mean()

In [ ]:
dict(n)

In [ ]:
a=titanic[(titanic.Sex == 'male') & (titanic.Pclass == 3)].fillna(0)


In [ ]:
a

In [ ]:
titanic[(titanic.Sex == 'male') & (titanic.Pclass == 3)]

In [ ]:
titanic.Fare.describe()

In [ ]:
class_groups.Fare.describe()

In [ ]:
class_groups.apply(lambda x: x.Fare.describe()) # Note column index name

In [ ]:
titanic.groupby("Sex").Parch.mean()

In [ ]:
class_groups.apply(lambda x: x.groupby("Sex").Parch.mean()) # Note column index name

In [ ]:
class_groups.apply(lambda x: x[x.Parch==0].groupby("Sex").size())

In [ ]:
titanic[titanic.Parch!=0].groupby(["Sex", "Embarked"]).size()

In [ ]:
class_groups.apply(lambda x: x[x.Parch!=0].groupby(["Sex", "Embarked"]).size())

In [ ]:
result = (class_groups
          .apply(lambda x: x[x.Parch!=0].groupby(["Sex", "Embarked"]).size())
          .unstack([1,2]))

In [ ]:
result

In [ ]:
result.loc[:, ("female", "C")]

In [ ]:
for gi, group in class_groups:
    print(group)

In [ ]:
result.loc[:, [("female", "C"), ("female", "S")]]

In [ ]:
result.iloc[:, [0, 1]]

# DataFrame output

In [ ]:
titanic[["SibSp", "Parch"]].head()

In [ ]:
(titanic[titanic.Parch!=0].groupby(["Sex", "Embarked"])[["SibSp", "Parch"]]
        .mean())

In [ ]:
(class_groups
 .apply(lambda x: x[x.Parch!=0].groupby(["Sex", "Embarked"])[["SibSp", "Parch"]]
        .mean()))

## Mixing group keys

In [ ]:
titanic.head()

In [ ]:
?titanic.set_index

In [ ]:
titanic_idx = titanic.reset_index().set_index((5 + 10*(titanic.Age//10)))

In [ ]:
titanic_idx.head()

In [ ]:
titanic_idx.groupby([pd.Grouper(level=0), "Pclass"]).Parch.mean()

In [ ]:
titanic_idx.groupby([titanic_idx.index, "Pclass"]).Parch.mean()

# pd.Grouper

In [ ]:
class_emb_groups = titanic.groupby([pd.Grouper("Embarked"), "Pclass"])

In [ ]:
class_emb_groups.size()

# Join operations

In [ ]:
a = pd.DataFrame(np.arange(8).reshape((4,2)),
                 columns=["a", "b"],
                 index=["a", "b", "a", "b"])
b = pd.DataFrame(10 + np.arange(4).reshape((4,-1)),
                 columns=["d"],
                 index=["d", "b", "c", "b"])

In [ ]:
print(a)
print(b)

In [ ]:
a.join(b) # default is left join

In [ ]:
a.join(b, how="inner")

In [ ]:
print(a)
print(b)

In [ ]:
a.join(b, how="right")

In [ ]:
a.join(b, how="outer", on="")

In [ ]:
c = pd.DataFrame(np.arange(8).reshape((4,2)),
                 columns=["a", "b"],
                 index=pd.MultiIndex.from_tuples([("a", "A"), ("b", "E"), ("a", "Y"), ("b", "R")], names=("lower", "upper")))

In [ ]:
c

In [ ]:
a.join?

In [ ]:
c.join(a[a.a<10], on="lower", rsuffix="_right")

In [ ]:
titanic.head()

In [ ]:
titanic.replace(re.compile(r'\s+\(.*\)'), '')

In [5]:
family_names = (titanic
                .replace(re.compile(r'\s+\(.*\)'), '')
                .replace(re.compile("Mrs."), "Mr."))[["Name", "Sex"]]

In [6]:
family_names["PassengerId"]

KeyError: 'PassengerId'

In [ ]:
titanic.head(5)

In [ ]:
family_names.head()

In [7]:
family_names = family_names[(family_names.Sex=="female") & family_names.Name.str.contains("Mr.")]

In [ ]:
family_names.head()

In [8]:
family_names.reset_index().set_index("Name")#["PassengerId"]

,PassengerId,Sex
Name,,
"Cumings, Mr. John Bradley",2,female
"Futrelle, Mr. Jacques Heath",4,female
"Johnson, Mr. Oscar W",9,female
"Nasser, Mr. Nicholas",10,female
"Hewlett, Mr.",16,female
"Vander Planke, Mr. Julius",19,female
"Masselmani, Mr. Fatima",20,female
"Asplund, Mr. Carl Oscar",26,female
"Spencer, Mr. William Augustus",32,female


In [9]:
titanic = (titanic.join(family_names
                        .reset_index()
                        .set_index("Name")["PassengerId"],
                        on="Name", rsuffix="_Spouse"))

In [ ]:
titanic.head(25)

In [ ]:
titanic.rename({"PassengerId":"PassengerId_Spouse"}, axis=1, inplace=True)

In [ ]:
titanic.head()

In [ ]:
?titanic.join

In [ ]:
titanic = titanic.join(titanic[["Name", "Age"]],
                       on="PassengerId_Spouse", rsuffix="_Spouse")

In [ ]:
titanic

In [ ]:
(titanic[titanic.PassengerId_Spouse.notnull()]
 [["Pclass", "Name", "Age", "Name_Spouse", "Age_Spouse"]])

In [ ]:
titanic.Pclass.value_counts()

In [ ]:
titanic[titanic.PassengerId_Spouse.notnull()].groupby("Pclass").size()

In [ ]:
titanic.Age - titanic.Age_Spouse

In [ ]:
(titanic.Age - titanic.Age_Spouse).groupby(titanic.Pclass).agg(["min", "max", "mean", "std", "count"])

In [ ]:
titanic.loc[742]

In [ ]:
(titanic.Age - titanic.Age_Spouse).sort_values()

In [ ]:
FTS_COLS = titanic.columns[1:-3].tolist()
FTS_COLS.remove("Cabin")

In [ ]:
titanic.info()

In [ ]:
titanic[FTS_COLS].isnull().any(axis=1).sum()

In [ ]:
titanic[(titanic.Age - titanic.Age_Spouse)<0][["Pclass", "Name", "Age", "Name_Spouse", "Age_Spouse"]]